In [ ]:
!pip install pandas
!pip install datasets,
!pip install pytorch-ignite
!pip install torchmetrics
!pip install pytorch-lightning
!pip install transformers

In [19]:
import ast
import pandas as pd
from eval_utils import rougel_eval, selfbleu_eval, bertscore_eval, bleurt_eval, concat_gen, tokenize
from gen_answerability import run_generate

In [43]:
TYPE2CONTROL_SIGNAL = ['What', 'How', 'Who', 'Why', 'Where', 'When', 'Which']

# path for generated result
result_dir = 'output/pred_q.csv'
result_df = pd.read_csv(result_dir)
for i in TYPE2CONTROL_SIGNAL:
    result_df[i] = result_df[i].apply(lambda x: ast.literal_eval(x))
    
# path for golden data -> original data(xlsx -> csv)
test_dir = 'data/ftqa_wh_2_test1.csv'
gold_df = pd.read_csv(test_dir)
gold_df = gold_df.loc[:,['cor_section', 'question']]
gold_df.columns = ['context', 'question']

# merge dataframe
gb = gold_df.groupby(['context'])
result = pd.DataFrame(gb['question'].unique())

tot_df = pd.merge(result, result_df, on="context")
tot_df = tot_df.dropna()
tot_df['tot_gen'] = tot_df.apply(lambda x: concat_gen(x, 4, TYPE2CONTROL_SIGNAL), axis=1)

In [47]:
tot_df.head(3)

,context,question,What,How,Who,Why,Where,When,Which,tot_gen
0,"""'God save you,' says the King (for the King w...",[Why was the king surprised Saint Kavin knew a...,"[What did Saint Kavin say he was?, What did Sa...","[How did Saint Kavin know about the goose?, Ho...","[Who told Saint Kavin about the goose?, Who di...",[Why did the king think Saint Kavin knew about...,"[Where did Saint Kavin live?, Where did Saint ...","[When did Saint Kavin learn about the goose?, ...",[Which one did Saint Kavin say told him about ...,"[What did Saint Kavin say he was?, What did Sa..."
1,"""'It is I that should kneel to you, Sweetheart...",[Why was the prince trapped as a yellow Paddoc...,[What did the Prince want to do after he broke...,[How will the prince feel when he meets the gi...,"[Who did the Prince cast the spell on?, Who ca...",[Why did the prince wait for a maiden to appea...,[Where did the prince wait for a maiden to app...,[When will the prince return and claim his fat...,[Which prince will be the next to claim his fa...,[What did the Prince want to do after he broke...
2,"""'It's well for you,' says Saint Kavin, mighty...",[What will happen if the king does not say tha...,"[What did the horse-eel do to the goose?, What...",[How did Saint Kavin feel when the king said h...,"[Who was the cute goose?, Who was let into the...","[Why didn't the king eat the goose?, Why did t...","[Where did the goose fly around?, Where did th...","[When did the goose catch a horse-eel?, When d...","[Which goose was killed by the horse-eel?, Whi...","[What did the horse-eel do to the goose?, What..."


In [55]:
rougel_eval(tot_df)

TypeError: ROUGEScore.update() missing 1 required positional argument: 'target'

In [56]:
selfbleu_eval(tot_df)

Self-BLEU: 0.6364276971459654


0.6364276971459654

In [57]:
bertscore_eval(tot_df)

/Users/eastshine/Documents/GitHub/youngyanggang/baselines/mQG/src/eval_utils.py:61: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  results = []
/Users/eastshine/Documents/GitHub/youngyanggang/.conda/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


ImportError: To be able to use bertscore, you need to install the following dependency: bert_score.
Please install it using 'pip install bert_score' for instance.

In [ ]:
bleurt_eval(tot_df)

In [ ]:
# answerability evaluation model checkpoint
ckpt_path = 'checkpoint/epoch=5-step=1109.ckpt'
    
ans_df = tot_df.loc[:, ['context', 'tot_gen']]
ans_df = ans_df.explode('tot_gen').reset_index(inplace=False, drop=True)

run_generate(ans_df, ckpt_path)